In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from keras import backend as K
import pickle as pkl
from timeit import default_timer as timer

import numpy as np
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from keras_helper import NNWeightHelper
from snes import SNES
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.examples.tutorials.mnist import input_data


In [3]:
SAMPLE_SIZE = 1024

POPULATION_SIZE = 10

GENERATIONS = 30

In [4]:
img_width, img_height = 150, 150
train_data_dir = "/home/river/assi2_ce888/Offical31_datasets/dslr/images"

validation_data_dir = "/home/river/assi2_ce888/Offical31_datasets/amazon/images"


In [5]:
nb_train_samples = 527
nb_validation_samples = 423
epochs = 10
batch_size = 128
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [6]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    #save_to_dir= "/home/river/assi2_ce888/Offical31_datasets/save_pic",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 498 images belonging to 31 classes.
Found 2817 images belonging to 31 classes.


In [45]:
len(train_generator)

4

In [46]:
x = train_generator[0][1]

In [32]:
len(x)

128

In [40]:
train = []
train_imgs = []
train_labels = []
for i in range(0,len(train_generator)):
    train_imgs.append(train[i][0])
    train_labels.append(train[i][1])

IndexError: list index out of range

In [24]:
len(train)

23

In [7]:
x_train , y_labels= train_generator.next()
x_test, test_labels= validation_generator.next()

150

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(31)) # the number o node was equal to the number of classes we want to recognize
model.add(Activation('softmax'))

In [9]:
model.compile(loss = "categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

In [51]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
4/4 [==============================] - 29s 7s/step - loss: 3.5252 - acc: 0.0345 - val_loss: 3.4493 - val_acc: 0.0339
Epoch 2/10
4/4 [==============================] - 24s 6s/step - loss: 3.4181 - acc: 0.0487 - val_loss: 3.4440 - val_acc: 0.0443
Epoch 3/10
4/4 [==============================] - 25s 6s/step - loss: 3.3991 - acc: 0.0399 - val_loss: 3.4647 - val_acc: 0.0234
Epoch 4/10
4/4 [==============================] - 25s 6s/step - loss: 3.3612 - acc: 0.0608 - val_loss: 3.4988 - val_acc: 0.0443
Epoch 5/10
4/4 [==============================] - 25s 6s/step - loss: 3.3255 - acc: 0.0799 - val_loss: 3.4888 - val_acc: 0.0391
Epoch 6/10
4/4 [==============================] - 25s 6s/step - loss: 3.2769 - acc: 0.0854 - val_loss: 3.5675 - val_acc: 0.0469
Epoch 7/10
4/4 [==============================] - 26s 7s/step - loss: 3.2145 - acc: 0.1013 - val_loss: 3.5038 - val_acc: 0.0286
Epoch 8/10
4/4 [==============================] - 28s 7s/step - loss: 3.1600 - acc: 0.1241 - val_loss: 3

In [ ]:

train_imgs = []
batch_index = 0

while batch_index <= train_generator.batch_index:
    train = train_generator.next() 
    train_imgs.append(train[0])
    batch_index = batch_index + 1
    

train_imags_array = np.asarray(train_imgs)

In [ ]:
test_imgs = []
batch_index = 0
while batch_index <= validation_generator.batch_index:
    test = validation_generator.next()
    test_imgs.append(test[0])
    batch_index = batch_index +1

test_imags_array = np.asarray(test_imgs)

In [ ]:
test_imags_array.shape

In [ ]:
train_imags_array.shape

In [ ]:
train_labels=train_generator.class_indices
test_labels= validation_generator.class_indices

In [ ]:
print(test_labels)

In [ ]:
train_imags_label = train_generator


In [ ]:
len(train_imags_label)

In [10]:
def train_classifier(model, X, y):
    X_features = model.predict(X)
    clf = ExtraTreesClassifier(n_estimators=100, n_jobs=4)
    #clf = DecisionTreeClassifier()
    clf.fit(X_features, y)
    y_pred = clf.predict(X_features)
    return clf, y_pred


def predict_classifier(model, clf, X):
    X_features = model.predict(X)
    return clf.predict(X_features)

In [11]:
nnw = NNWeightHelper(model)
weights = nnw.get_weights()


In [ ]:
print(weights.shape)

In [54]:
all_examples_indices = list(range(x_train.shape[0]))

In [12]:

clf, _ = train_classifier(model, x_train, y_labels)

In [13]:
print("x_train shape is :", x_train.shape)
print("combined_train_labels shape is :",y_labels.shape)
y_pred = predict_classifier(model, clf, x_test)

x_train shape is : (128, 150, 150, 3)
combined_train_labels shape is : (128, 31)


In [14]:
print(test_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


print(test_labels.shape, y_pred.shape)
test_accuracy = accuracy_score(test_labels, y_pred)

print('Non-trained NN Test accuracy:', test_accuracy)
snes = SNES(weights, 1, POPULATION_SIZE)


In [15]:
for i in range(0, GENERATIONS):
    start = timer()
    asked = snes.ask()

    # to be provided back to snes
    told = []
    # use a small number of training samples for speed purposes
    subsample_indices = np.random.choice(all_examples_indices, size=SAMPLE_SIZE)#, replace=False)
    # evaluate on another subset
    subsample_indices_valid = np.random.choice(all_examples_indices, size=SAMPLE_SIZE + 1)#, replace=False)
    #subsample_indices_valid  = mnistm_valid
    # iterate over the population
    for asked_j in asked:
        nnw.set_weights(asked_j)
        # set nn weights
        # train the classifer and get back the predictions on the training data
        clf, _ = train_classifier(model, x_train[subsample_indices], y_labels[subsample_indices])

        # calculate the predictions on a different set
        y_pred = predict_classifier(model, clf, x_train[subsample_indices_valid])
        score = accuracy_score(y_labels[subsample_indices_valid], y_pred)

        #clf, _ = train_classifier(model, combined_train_imgs, combined_train_domain_class)
        #y_pred = predict_classifier(model, clf, combined_test_imgs)
        #score = accuracy_score(combined_test_domain_class, y_pred)
        # append to array of values that are to be returned
        told.append(score)

    snes.tell(asked, told)
    end = timer()
    print("It took", end - start, "seconds to complete generation", i + 1)

nnw.set_weights(snes.center)

clf, _ = train_classifier(model, x_train,y_labels)
y_pred = predict_classifier(model, clf, x_test,test_labels)

print(combined_test_domain_class.shape, y_pred.shape)
test_accuracy = accuracy_score(test_labels, y_pred)
print('Test accuracy:', test_accuracy)

NameError: name 'snes' is not defined